In [1]:
# Parameters
RUN_DATE = "2025-10-25"


<a href="https://colab.research.google.com/github/HieuNguyenPhi/ADJ_JOBS/blob/main/notebooks/ADJUST_JOB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UAT

In [2]:
import os
from azure.storage.blob import BlobServiceClient

account_name = os.getenv('ACCOUNT_NAME')
account_key = os.getenv('ACCOUNT_KEY')
# Replace with your Azure Storage account name and SAS token or connection string
connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_list = blob_service_client.list_containers()
container_name = "adjuststbuatprocessed" #os.getenv('CONTAINER_NAME')
container_client = blob_service_client.get_container_client(container_name)
# already_processed = [file.name.split('/')[1].split('.')[0] for file in container_client.list_blobs() if file.name.split('/')[0] == 'output']
# already_processed[-5:]
already_processed_ts = sorted([file.name.split('/')[-1].split(".")[0] for file in container_client.list_blobs() if file.name.split('/')[0] == 'processing'])
already_processed_ts[-5:]

['2025-10-23T200000',
 '2025-10-23T210000',
 '2025-10-23T220000',
 '2025-10-23T230000',
 '2025-10-24T000000']

In [3]:
container_name_uat = "adjuststbuat"
container_client_uat = blob_service_client.get_container_client(container_name_uat)
from collections import defaultdict
files = [i.name for i in container_client_uat.list_blobs()]
groups = defaultdict(list)
for f in files:
    dt = f.split('_')[1]
    groups[dt].append(f)
groups[dt]

['rsh20bkkb4zk_2025-10-24T230000_762c775ae454d23f2c6b6a75623d14c7_be8220.csv.gz']

In [4]:
# from datetime import date, timedelta, datetime
# import pandas as pd
# today = date.today().strftime('%Y-%m-%d')
# yesterday = (date.today() - timedelta(days = 1) ).strftime('%Y-%m-%d')
# check_date = dt.split("T")[0]
# if check_date == today:
#     need_process = pd.date_range(start=already_processed[-1], end=today).strftime('%Y-%m-%d').to_list()
# else:
#     need_process = pd.date_range(start=already_processed[-1], end=yesterday).strftime('%Y-%m-%d').to_list()
# need_process

In [5]:
from datetime import datetime
import pandas as pd
B = datetime.strptime(dt, "%Y-%m-%dT%H0000")
A = datetime.strptime(already_processed_ts[-2], "%Y-%m-%dT%H0000")
need_process_ts =  pd.date_range(A, B, freq='h').strftime('%Y-%m-%dT%H0000').tolist()
need_process_ts

['2025-10-23T230000',
 '2025-10-24T000000',
 '2025-10-24T010000',
 '2025-10-24T020000',
 '2025-10-24T030000',
 '2025-10-24T040000',
 '2025-10-24T050000',
 '2025-10-24T060000',
 '2025-10-24T070000',
 '2025-10-24T080000',
 '2025-10-24T090000',
 '2025-10-24T100000',
 '2025-10-24T110000',
 '2025-10-24T120000',
 '2025-10-24T130000',
 '2025-10-24T140000',
 '2025-10-24T150000',
 '2025-10-24T160000',
 '2025-10-24T170000',
 '2025-10-24T180000',
 '2025-10-24T190000',
 '2025-10-24T200000',
 '2025-10-24T210000',
 '2025-10-24T220000',
 '2025-10-24T230000']

In [6]:
import polars as pl 
from tqdm import tqdm
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}

for ts, files in tqdm(groups.items()):
    if ts not in need_process_ts:
        continue
    dt = ts[:10]
    # if dt not in need_process:
    #     continue
    df = pl.scan_csv(f"az://adjuststbuat/*_{ts}_*.csv.gz", storage_options = storage_options,glob=True, has_header = True, null_values = ["","NULL"], ignore_errors=True).select(pl.all().cast(pl.Utf8))
    df.sink_parquet(f"az://adjuststbuatprocessed/processing/dt={dt}/{ts}.parquet", storage_options = storage_options, compression="snappy")
    print(f'Done dt={dt}/{ts}.parquet')
        

  0%|          | 0/2159 [00:00<?, ?it/s]

 99%|█████████▉| 2143/2159 [00:09<00:00, 216.90it/s]

Done dt=2025-10-23/2025-10-23T230000.parquet


Done dt=2025-10-24/2025-10-24T000000.parquet


 99%|█████████▉| 2143/2159 [00:19<00:00, 216.90it/s]

...

Done dt=2025-10-24/2025-10-24T130000.parquet


100%|█████████▉| 2158/2159 [02:11<00:00,  1.81it/s]

Done dt=2025-10-24/2025-10-24T140000.parquet


100%|██████████| 2159/2159 [02:18<00:00, 15.56it/s]

Done dt=2025-10-24/2025-10-24T230000.parquet


In [7]:
need_process = set([i.split("T")[0] for i in need_process_ts])
need_process

{'2025-10-23', '2025-10-24'}

In [8]:
for dt in need_process:
  df = pl.scan_parquet(f"az://adjuststbuatprocessed/processing/dt={dt}/*.parquet", storage_options=storage_options,glob=True).with_columns(pl.lit(dt).alias("dt"))
  df.sink_parquet(f"az://adjuststbuatprocessed/output/{dt}.parquet", storage_options=storage_options, compression="snappy")
  print(f'\n Done {dt}\n')


 Done 2025-10-24




 Done 2025-10-23



# Live

In [9]:
# already_processed = [file.name.split('/')[-1].split('.')[0] for file in container_client.list_blobs() if file.name[:12] == 'live/output/']
# already_processed[-5:]
already_processed_ts = sorted([file.name.split('/')[-1].split(".")[0] for file in container_client.list_blobs() if (file.name.split('/')[0] + "/" + file.name.split('/')[1]) == 'live/processing'])
already_processed_ts[-5:]

['2025-10-23T200000',
 '2025-10-23T210000',
 '2025-10-23T220000',
 '2025-10-23T230000',
 '2025-10-24T000000']

In [10]:
container_name_uat = "adjuststblive"
container_client_uat = blob_service_client.get_container_client(container_name_uat)
from collections import defaultdict
files = [i.name for i in container_client_uat.list_blobs()]
groups = defaultdict(list)
for f in files:
    dt = f.split('_')[1]
    groups[dt].append(f)
groups[dt]

['65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_2853a0.csv.gz',
 '65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_2853a1.csv.gz',
 '65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_be8220.csv.gz',
 '65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_be8221.csv.gz',
 '65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_c35750.csv.gz',
 '65n1fgov4zr4_2025-10-25T000000_762c775ae454d23f2c6b6a75623d14c7_c35751.csv.gz']

In [11]:
# need_process = pd.date_range(start=already_processed[-1], end=today).strftime('%Y-%m-%d').to_list()
# need_process

B = datetime.strptime(dt, "%Y-%m-%dT%H0000")
A = datetime.strptime(already_processed_ts[-1], "%Y-%m-%dT%H0000")
need_process_ts =  pd.date_range(A, B, freq='h').strftime('%Y-%m-%dT%H0000').tolist()
need_process_ts

['2025-10-24T000000',
 '2025-10-24T010000',
 '2025-10-24T020000',
 '2025-10-24T030000',
 '2025-10-24T040000',
 '2025-10-24T050000',
 '2025-10-24T060000',
 '2025-10-24T070000',
 '2025-10-24T080000',
 '2025-10-24T090000',
 '2025-10-24T100000',
 '2025-10-24T110000',
 '2025-10-24T120000',
 '2025-10-24T130000',
 '2025-10-24T140000',
 '2025-10-24T150000',
 '2025-10-24T160000',
 '2025-10-24T170000',
 '2025-10-24T180000',
 '2025-10-24T190000',
 '2025-10-24T200000',
 '2025-10-24T210000',
 '2025-10-24T220000',
 '2025-10-24T230000',
 '2025-10-25T000000']

In [12]:
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}

for ts, files in tqdm(groups.items()):
    if ts not in need_process_ts: continue
    dt = ts[:10]
    # if dt not in need_process:
    #     continue
    df = pl.scan_csv(f"az://adjuststblive/*_{ts}_*.csv.gz", storage_options = storage_options,glob=True, has_header = True, null_values = ["","NULL"], ignore_errors=True).select(pl.all().cast(pl.Utf8))
    df.sink_parquet(f"az://adjuststbuatprocessed/live/processing/dt={dt}/{ts}.parquet", storage_options = storage_options, compression="snappy")
    print(f'Done dt={dt}/{ts}.parquet')
        

  0%|          | 0/2223 [00:00<?, ?it/s]

 99%|█████████▉| 2199/2223 [00:23<00:00, 93.70it/s]

Done dt=2025-10-24/2025-10-24T000000.parquet


 99%|█████████▉| 2199/2223 [00:35<00:00, 93.70it/s]

 99%|█████████▉| 2200/2223 [00:43<00:00, 42.33it/s]

...

100%|█████████▉| 2222/2223 [07:40<00:12, 12.11s/it]

Done dt=2025-10-24/2025-10-24T230000.parquet


100%|██████████| 2223/2223 [07:59<00:00, 14.17s/it]

100%|██████████| 2223/2223 [07:59<00:00,  4.63it/s]

Done dt=2025-10-25/2025-10-25T000000.parquet


In [13]:
need_process = set([i.split("T")[0] for i in need_process_ts])
need_process

{'2025-10-24', '2025-10-25'}

In [14]:
for dt in need_process:
  df = pl.scan_parquet(f"az://adjuststbuatprocessed/live/processing/dt={dt}/*.parquet", storage_options=storage_options,glob=True).with_columns(pl.lit(dt).alias("dt"))
  df.sink_parquet(f"az://adjuststbuatprocessed/live/output/{dt}.parquet", storage_options=storage_options, compression="snappy")
  print(f'\n Done {dt}\n')


 Done 2025-10-24




 Done 2025-10-25

